In [3]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split

# 路径配置
excel_path = '241110 pym自评label-et.xlsx'  # Excel文件路径
image_folder = 'ET分级大圆修改'  # 原始图片文件夹路径
target_folder = 'ET分级大圆new' # 目标文件夹路径（train和test文件夹将在此创建）
df = pd.read_excel(excel_path)
df['ID'] = df['Image'].apply(lambda x: x.split('-')[1])
train_df, test_df = train_test_split(df, test_size=0.4, stratify=df['ID'], random_state=42)
def prepare_folders_and_copy_files(subset_df, subset_name):
    for _, row in subset_df.iterrows():
        file_name, category, id_ = row['Image'], str(row['Large circle']), row['ID']
        src_file_path = os.path.join(image_folder, file_name)
        target_dir_path = os.path.join(target_folder, subset_name, category)
        
        if not os.path.exists(target_dir_path):
            os.makedirs(target_dir_path)
        
        shutil.copy(src_file_path, target_dir_path)
prepare_folders_and_copy_files(train_df, 'train')
prepare_folders_and_copy_files(test_df, 'test')

print('图片已成功分配到训练集和测试集！')

图片已成功分配到训练集和测试集！


In [5]:
import imgaug as ia
import imgaug.augmenters as iaa
import os
from PIL import Image
import numpy as np

def augment_image(image_path, augmenter, output_dir, max_images):
    image = Image.open(image_path)
    image = np.array(image)
    aug_image = augmenter.augment_image(image)
    aug_image_pil = Image.fromarray(aug_image)
    output_path = os.path.join(output_dir, f"aug_{max_images}.png")
    aug_image_pil.save(output_path)
    return output_path

def augment_images_in_folder(folder_path, augmenter, target_total=800):
    original_images = [f for f in os.listdir(folder_path) if f.endswith('.png') or f.endswith('.jpg')]
    total_images = len(original_images)

    if total_images >= target_total:
        return  
    images_per_original = (target_total - total_images) // total_images
    for i, image_file in enumerate(original_images):
        image_path = os.path.join(folder_path, image_file)
        for _ in range(images_per_original):
            if total_images >= target_total:
                break
            augment_image(image_path, augmenter, folder_path, total_images)
            total_images += 1

def augment_dataset(base_folder):
    # 定义一系列图像增强操作
    augmenter = iaa.Sequential([
        iaa.Affine(rotate=(-25, 25), scale=(0.8, 1.2)),
        iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),
        iaa.GaussianBlur(sigma=(0, 3.0)),
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.LinearContrast((0.75, 1.5)),
        iaa.Multiply((0.8, 1.2)),
        iaa.Crop(percent=(0, 0.1))
    ])

    for category_folder in os.listdir(base_folder):
        category_path = os.path.join(base_folder, category_folder)
        if os.path.isdir(category_path):
            augment_images_in_folder(category_path, augmenter)

base_folder = "ET分级大圆new/train"  
augment_dataset(base_folder)